In [240]:
import os 
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef

path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"


***
### Matrice results: 

In [242]:
# TropiGAT
tropigat_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiGAT.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropigat_pred_m_df["protein_id"] = tropigat_pred_m_df["protein"].apply(lambda x : x.split("_Dpo")[0])
tropigat_pred_m_df.columns = ["protein_TropiGAT", "prediction_TropiGAT", "protein_id"]

tropigat_pred_m_df_new = pd.read_csv(f"{path_benchmark}/TropiGAT.review.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropigat_pred_m_df_new["protein_id"] = tropigat_pred_m_df_new["protein"].apply(lambda x : x.split("_Dpo")[0])
tropigat_pred_m_df_new.columns = ["protein_TropiGAT_new", "prediction_TropiGAT_new", "protein_id"]
tropigat_pred_m_df_new = tropigat_pred_m_df_new[["prediction_TropiGAT_new", "protein_id"]]

# TropiSAGE
tropisage_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiSAGE.review.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropisage_pred_m_df["protein_id"] = tropisage_pred_m_df["protein"].apply(lambda x : x.split("_Dpo")[0])
tropisage_pred_m_df.columns = ["protein_TropiSAGE", "prediction_TropiSAGE", "protein_id"]
tropisage_pred_m_df = tropisage_pred_m_df[["prediction_TropiSAGE", "protein_id"]]

# TropiSEQ
tropiseq_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiSEQ.p_05.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropiseq_pred_m_df["protein_id_1"] = tropiseq_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
tropiseq_pred_m_df["protein_id"] = tropiseq_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
tropiseq_pred_m_df.columns = ["protein_TropiSEQ", "prediction_TropiSEQ", "trash_seq" ,"protein_id"]
tropiseq_pred_m_df = tropiseq_pred_m_df[["prediction_TropiSEQ" ,"protein_id"]]

# TropiLR
tropilr_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropilr_pred_m_df["protein_id_1"] = tropilr_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
tropilr_pred_m_df["protein_id"] = tropilr_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
tropilr_pred_m_df.columns = ["protein_TropiLR", "prediction_TropiLR", "trash_seq" ,"protein_id"]
tropilr_pred_m_df = tropilr_pred_m_df[["prediction_TropiLR" ,"protein_id"]]

# SpikeHunter
SH_pred_m_df = pd.read_csv(f"{path_benchmark}/SpikeHunter_predictions.matrices.tsv", sep = "\t", names = ["protein", "prediction"])

SH_pred_m_df["protein_id_1"] = SH_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
SH_pred_m_df["protein_id"] = SH_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
SH_pred_m_df.columns = ["protein_SpikeHunter", "prediction_SpikeHunter", "trash_seq" ,"protein_id"]
SH_pred_m_df = SH_pred_m_df[["prediction_SpikeHunter" ,"protein_id"]]


# Merge : 
merged_df = tropigat_pred_m_df.merge(tropiseq_pred_m_df, on='protein_id', how='inner').merge(SH_pred_m_df, on='protein_id', how='inner').merge(tropigat_pred_m_df_new, on='protein_id', how='inner').merge(tropisage_pred_m_df, on='protein_id', how='inner').merge(tropilr_pred_m_df, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])


In [243]:
merged_df

,protein_TropiGAT,prediction_TropiGAT,protein_id,prediction_TropiSEQ,prediction_SpikeHunter,prediction_TropiGAT_new,prediction_TropiSAGE,prediction_TropiLR,phage
0,K15PH90__cds_54_Dpo_domain,KL123:0.997 ; KL27:0.988 ; KL112:0.985 ; KL14:...,K15PH90__cds_54,No_hits,K106,KL112:0.988 ; KL166:0.968 ; KL8:0.953 ; KL38:0...,KL166:1.0 ; KL112:1.0 ; KL8:0.999 ; KL124:0.99...,No_hits,K15PH90
1,K7PH164C4__cds_20_Dpo_domain,KL39:1.0 ; KL123:0.999 ; KL22:0.996 ; KL114:0....,K7PH164C4__cds_20,No_hits,No_hits,KL105:1.0 ; KL149:0.999 ; KL22:0.998 ; KL106:0...,KL47:1.0 ; KL116:1.0 ; KL105:1.0 ; KL39:1.0 ; ...,No_hits,K7PH164C4
2,K32PH164C1__cds_20_Dpo_domain,KL36:1.0 ; KL39:0.998 ; KL25:0.992 ; KL116:0.9...,K32PH164C1__cds_20,No_hits,No_hits,KL36:1.0 ; KL105:0.998 ; KL8:0.998 ; KL22:0.99...,KL36:1.0 ; KL166:1.0 ; KL46:1.0 ; KL105:1.0 ; ...,No_hits,K32PH164C1
3,K18PH07C1__cds_245_Dpo_domain,KL3:0.998 ; KL63:0.984 ; KL43:0.94 ; KL145:0.9...,K18PH07C1__cds_245,KL63:0.594 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,K63,KL3:0.93 ; KL63:0.872 ; KL70:0.768 ; KL54:0.64...,KL6:1.0 ; KL63:0.999 ; KL26:0.988 ; KL31:0.974...,KL6:0.523 ; KL63:0.682,K18PH07C1
4,K13PH07C1L__cds_11_Dpo_domain,KL23:0.992 ; KL10:0.99 ; KL30:0.975 ; KL60:0.9...,K13PH07C1L__cds_11,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,K2,KL12:0.999 ; KL128:0.991 ; KL105:0.962 ; KL36:...,KL105:1.0 ; KL128:0.995 ; KL30:0.995 ; KL36:0....,No_associations,K13PH07C1L
...,...,...,...,...,...,...,...,...,...
259,PFOEGONH_00078,KL3:1.0 ; KL81:0.985 ; KL14:0.957 ; KL52:0.857...,PFOEGONH_00078,KL35:0.736 ; KL3:0.674 ; KL124:0.332 ; KL37:0....,K3,KL3:0.999 ; KL117:0.951 ; KL31:0.789 ; KL52:0....,KL3:1.0 ; KL67:0.99 ; KL14:0.862 ; KL52:0.794 ...,KL35:0.71 ; KL3:0.947,PFOEGONH
260,NBNDMPCG_00163,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...,NBNDMPCG_00163,KL2:0.684 ; KL52:0.239 ; KL37:0.235 ; KL164:0....,K2,KL12:0.998 ; KL13:0.929 ; KL9:0.92 ; KL2:0.652...,KL12:0.999 ; KL9:0.99 ; KL13:0.981 ; KL142:0.9...,KL2:0.829 ; KL52:0.774,NBNDMPCG
261,NJHLHPIG_00061,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....,NJHLHPIG_00061,KL46:0.937 ; KL18:0.522 ; KL37:0.235 ; KL164:0...,K46,KL128:0.999 ; KL46:0.999 ; KL12:0.984 ; KL56:0...,KL46:0.999 ; KL56:0.998 ; KL128:0.984 ; KL26:0...,KL46:0.82,NJHLHPIG
262,HIIECEMK_00054,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....,HIIECEMK_00054,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,K2,KL12:0.991 ; KL18:0.976 ; KL60:0.93 ; KL70:0.7...,KL8:0.999 ; KL12:0.998 ; KL18:0.997 ; KL142:0....,No_associations,HIIECEMK


In [244]:
missing_prot = [prot for prot in list(tropigat_pred_m_df["protein_id"]) if prot not in list(SH_pred_m_df["protein_id"])]
missing_prot_prime = [prot for prot in list(SH_pred_m_df["protein_id"]) if prot not in list(tropigat_pred_m_df["protein_id"])]

In [245]:
missing_prot

['BLCJPOBP_00052']

In [246]:
missing_prot_prime

['K33PH14C2__cds_55', 'K62PH164C2__cds_25']

In [247]:
len(tropigat_pred_m_df), len(tropiseq_pred_m_df),len(SH_pred_m_df), len(merged_df)

(258, 260, 260, 264)

In [248]:
phages_to_eliminate = ["K80PH1317b", "K80PH1317a"] + ["K2064PH2","K2069PH1","OBHDAGOG","A1e","A1f","A3a","EONHMLJF"]
merged_df_clean_01 = merged_df[["phage", "protein_id", "prediction_TropiGAT", "prediction_TropiGAT_new", "prediction_TropiSAGE", "prediction_TropiSEQ", "prediction_TropiLR", "prediction_SpikeHunter"]]
#merged_df_clean_01.columns = ["phage", "protein", "TropiGAT_prediction", "TropiSEQ_prediction", "SpikeHunter_prediction"]
merged_df_clean_01 = merged_df_clean_01[~merged_df_clean_01["phage"].isin(phages_to_eliminate)] 
merged_df_clean_01

,phage,protein_id,prediction_TropiGAT,prediction_TropiGAT_new,prediction_TropiSAGE,prediction_TropiSEQ,prediction_TropiLR,prediction_SpikeHunter
0,K15PH90,K15PH90__cds_54,KL123:0.997 ; KL27:0.988 ; KL112:0.985 ; KL14:...,KL112:0.988 ; KL166:0.968 ; KL8:0.953 ; KL38:0...,KL166:1.0 ; KL112:1.0 ; KL8:0.999 ; KL124:0.99...,No_hits,No_hits,K106
1,K7PH164C4,K7PH164C4__cds_20,KL39:1.0 ; KL123:0.999 ; KL22:0.996 ; KL114:0....,KL105:1.0 ; KL149:0.999 ; KL22:0.998 ; KL106:0...,KL47:1.0 ; KL116:1.0 ; KL105:1.0 ; KL39:1.0 ; ...,No_hits,No_hits,No_hits
2,K32PH164C1,K32PH164C1__cds_20,KL36:1.0 ; KL39:0.998 ; KL25:0.992 ; KL116:0.9...,KL36:1.0 ; KL105:0.998 ; KL8:0.998 ; KL22:0.99...,KL36:1.0 ; KL166:1.0 ; KL46:1.0 ; KL105:1.0 ; ...,No_hits,No_hits,No_hits
3,K18PH07C1,K18PH07C1__cds_245,KL3:0.998 ; KL63:0.984 ; KL43:0.94 ; KL145:0.9...,KL3:0.93 ; KL63:0.872 ; KL70:0.768 ; KL54:0.64...,KL6:1.0 ; KL63:0.999 ; KL26:0.988 ; KL31:0.974...,KL63:0.594 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,KL6:0.523 ; KL63:0.682,K63
4,K13PH07C1L,K13PH07C1L__cds_11,KL23:0.992 ; KL10:0.99 ; KL30:0.975 ; KL60:0.9...,KL12:0.999 ; KL128:0.991 ; KL105:0.962 ; KL36:...,KL105:1.0 ; KL128:0.995 ; KL30:0.995 ; KL36:0....,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,No_associations,K2
...,...,...,...,...,...,...,...,...
259,PFOEGONH,PFOEGONH_00078,KL3:1.0 ; KL81:0.985 ; KL14:0.957 ; KL52:0.857...,KL3:0.999 ; KL117:0.951 ; KL31:0.789 ; KL52:0....,KL3:1.0 ; KL67:0.99 ; KL14:0.862 ; KL52:0.794 ...,KL35:0.736 ; KL3:0.674 ; KL124:0.332 ; KL37:0....,KL35:0.71 ; KL3:0.947,K3
260,NBNDMPCG,NBNDMPCG_00163,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...,KL12:0.998 ; KL13:0.929 ; KL9:0.92 ; KL2:0.652...,KL12:0.999 ; KL9:0.99 ; KL13:0.981 ; KL142:0.9...,KL2:0.684 ; KL52:0.239 ; KL37:0.235 ; KL164:0....,KL2:0.829 ; KL52:0.774,K2
261,NJHLHPIG,NJHLHPIG_00061,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....,KL128:0.999 ; KL46:0.999 ; KL12:0.984 ; KL56:0...,KL46:0.999 ; KL56:0.998 ; KL128:0.984 ; KL26:0...,KL46:0.937 ; KL18:0.522 ; KL37:0.235 ; KL164:0...,KL46:0.82,K46
262,HIIECEMK,HIIECEMK_00054,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....,KL12:0.991 ; KL18:0.976 ; KL60:0.93 ; KL70:0.7...,KL8:0.999 ; KL12:0.998 ; KL18:0.997 ; KL142:0....,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,No_associations,K2


> Add the target data: 

In [249]:
# Fold data
path_project = "/media/concha-eloko/Linux/PPT_clean"
df_folds = pd.read_csv(f"{path_project}/in_vitro/dpos_folds.all_matrices.tsv", header = 0, sep = "\t")

# Target data
path_finetuning = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning"
bea_df = pd.read_csv(f"{path_finetuning}/bea_fine_tuning.df", sep = "\t", header = 0)
bea_df["Protein"] = bea_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_bea = set([kltype.strip() for kltypes in bea_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

ferriol_df = pd.read_csv(f"{path_finetuning}/ferriol_fine_tuning.df", sep = "\t", header = 0)
#ferriol_df["Target"] = ferriol_df["Target"].apply(lambda x : x.replace("K", "KL"))
pool_ferriol = set([kltype.strip() for kltypes in ferriol_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

towndsend_df = pd.read_csv(f"{path_finetuning}/towndsend_fine_tuning.df", sep = "\t", header = 0)
towndsend_df["Protein"] = towndsend_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_towndsend = set([kltype.strip() for kltypes in towndsend_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

dico_matrices = {"ferriol" : {"matrix" : ferriol_df, "pool" : pool_ferriol}, 
                 "bea" : {"matrix": bea_df, "pool" : pool_bea}, 
                 "towndsend" : {"matrix" : towndsend_df, "pool" : pool_towndsend}}

# targets dico : 
dico_hits = {}
for author in dico_matrices :
    matrix = dico_matrices[author]["matrix"]
    for _, row in matrix.iterrows() : 
        for phage in matrix["Phages"].unique() : 
            all_targets = set()
            targets = matrix[matrix["Phages"] == phage]["Target"].values
            for calls in targets : 
                actual_targets = [x.strip() for x in calls.split(",")]
                all_targets.update(actual_targets)
            dico_hits[phage] = all_targets


In [55]:
df_folds["protein_id"].tolist()

['A1a_00002',
 'A1a_00014',
 'A1b_00048',
 'A1b_00036',
 'A1c_00046',
 'A1c_00034',
 'A1d_00013',
 'A1d_00009',
 'A1e_00024',
 'A1e_00012',
 'A1f_00024',
 'A1f_00012',
 'A1g_00045',
 'A1g_00057',
 'A1h_00021',
 'A1h_00009',
 'A1h_00013',
 'A1i_00041',
 'A1i_00037',
 'A1i_00049',
 'A1j_00040',
 'A1j_00049',
 'A1j_00002',
 'A1k_00018',
 'A1k_00014',
 'A1l_00058',
 'A1l_00005',
 'A1m_00045',
 'A1m_00049',
 'A1n_00050',
 'A1o_00045',
 'A1o_00041',
 'A1p_00055',
 'A1q_00023',
 'A1q_00010',
 'A1q_00019',
 'A1r_00013',
 'A1r_00009',
 'A2a_00010',
 'A2a_b_00022',
 'A2a_b_00036',
 'A2a_00049',
 'A2b_00022',
 'A2b_00008',
 'A3a_00002',
 'A3a_00045',
 'A3b_00021',
 'A3b_00016',
 'A3c_00044',
 'A3c_00039',
 'A3c_00045',
 'A3d_00041',
 'A3d_00042',
 'A3d_00036',
 'BLCJPOBP_00041',
 'D7b_00043',
 'D7c_00007',
 'DIMCIIMF_00039',
 'DIMCIIMF_00240',
 'DJLANJJD_00238',
 'EHPPICDA_00095',
 'EKPIEFBL_00113',
 'EKPIEFBL_00177',
 'EONHMLJF_00087',
 'FADJDIKG_00083',
 'FADJDIKG_00019',
 'GCLPFEGH_00240',
 'H

In [250]:
# Initialize new columns :
target_serie = []
fold_serie = []
tropigat_pred_serie, tropigatnew_pred_serie, tropiseq_pred_serie, sh_pred_serie, tropilr_pred_serie, tropisage_pred_serie = [], [], [] , [], [] , []

# Feed comumns : 
for _, row in merged_df_clean_01.iterrows() :
    if row["phage"] not in dico_hits :
        print(f"weird phage: {row['phage']}")
    else :
        targets = dico_hits[row["phage"]]
        target_serie.append(",".join(targets))
        prot_it = row['protein_id'].replace("__cds", "_cds")
        try :
            fold = df_folds[df_folds["protein_id"] == prot_it]["Fold"].values[0]
        except IndexError  :
            fold = df_folds[df_folds["protein_id"] == row['protein_id']]["Fold"].values[0]
        except Exception as e :
            fold = "unknown"
        fold_serie.append(fold)
        
        # TropiGAT part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiGAT"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropigat_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropigat_pred_serie.append(0)
            
        # TropiGAT_new part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiGAT_new"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropigatnew_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropigatnew_pred_serie.append(0)
            
        # TropiSAGE part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiSAGE"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropisage_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropisage_pred_serie.append(0)
            
        # TropiSEQ part : 
        if row["prediction_TropiSEQ"][0] != "N" : 
            tropiseq_pred = [x.split(":")[0].strip() for x in row["prediction_TropiSEQ"].split(";")]
            top_predictions = set(tropiseq_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                tropiseq_pred_serie.append(",".join(list(good_calls)))
            else :
                tropiseq_pred_serie.append(0)
        else :
            tropiseq_pred_serie.append(0)
            
        # TropiLR part : 
        if row["prediction_TropiLR"][0] != "N" : 
            tropiseq_pred = [x.split(":")[0].strip() for x in row["prediction_TropiLR"].split(";")]
            top_predictions = set(tropiseq_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                tropilr_pred_serie.append(",".join(list(good_calls)))
            else :
                tropilr_pred_serie.append(0)
        else :
            tropilr_pred_serie.append(0)
            
        # SH part : 
        if row["prediction_SpikeHunter"][0] != "N" : 
            sh_pred = [x.split(":")[0].strip().replace("K", "KL") for x in row["prediction_SpikeHunter"].split(";")]
            top_predictions = set(sh_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                sh_pred_serie.append(",".join(list(good_calls)))
            else :
                sh_pred_serie.append(0)
        else :
            sh_pred_serie.append(0)
        
    

In [251]:
merged_df_clean_01["TropiGAT_good_calls"] = tropigat_pred_serie
merged_df_clean_01["TropiGAT_new_good_calls"] = tropigatnew_pred_serie
merged_df_clean_01["TropiSAGE_good_calls"] = tropisage_pred_serie

merged_df_clean_01["TropiSEQ_good_calls"] = tropiseq_pred_serie
merged_df_clean_01["TropiLogReg_good_calls"] = tropilr_pred_serie
merged_df_clean_01["SpikeHunter_good_calls"] = sh_pred_serie


merged_df_clean_01["Targets"] = target_serie
merged_df_clean_01["Folds"] = fold_serie

merged_df_clean_01.columns  = ['phage', 'protein', 'TropiGAT_prediction', 'TropiGAT_new_prediction',
       'TropiSAGE_prediction', 'TropiSEQ_prediction', 'TropiLogReg_prediction',
       'SpikeHunter_prediction', 'TropiGAT_good_calls',
       'TropiGAT_new_good_calls', 'TropiSAGE_good_calls',
       'TropiSEQ_good_calls', 'TropiLogReg_good_calls',
       'SpikeHunter_good_calls', 'Targets', 'Folds']

In [252]:
merged_df_clean_02 = merged_df_clean_01[["phage", "protein", "Folds", "TropiGAT_prediction", "TropiGAT_good_calls", "TropiGAT_new_prediction", "TropiGAT_new_good_calls", "TropiSAGE_prediction", "TropiSAGE_good_calls" , "TropiSEQ_prediction", "TropiSEQ_good_calls", "TropiLogReg_prediction", "TropiLogReg_good_calls" ,"SpikeHunter_prediction", "SpikeHunter_good_calls", "Targets"]]
merged_df_clean_02.sort_values(by='phage', ascending=True)
merged_df_clean_02.to_csv(f"{path_benchmark}/Prediction_results.v2.matrices.tsv", sep = "\t", header = True, index = False)


***
### Get metrics:

In [254]:
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter 

path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"

results_bench_df = pd.read_csv(f"{path_benchmark}/Prediction_results.v2.matrices.tsv", sep = "\t", index_col = False, header = 0)


In [261]:
results_bench_df

,phage,protein,Folds,TropiGAT_prediction,TropiGAT_good_calls,TropiGAT_new_prediction,TropiGAT_new_good_calls,TropiSAGE_prediction,TropiSAGE_good_calls,TropiSEQ_prediction,TropiSEQ_good_calls,TropiLogReg_prediction,TropiLogReg_good_calls,SpikeHunter_prediction,SpikeHunter_good_calls,Targets
0,K15PH90,K15PH90__cds_54,6-bladed beta-propeller,KL123:0.997 ; KL27:0.988 ; KL112:0.985 ; KL14:...,0,KL112:0.988 ; KL166:0.968 ; KL8:0.953 ; KL38:0...,0,KL166:1.0 ; KL112:1.0 ; KL8:0.999 ; KL124:0.99...,0,No_hits,0,No_hits,0,K106,0,KL15
1,K7PH164C4,K7PH164C4__cds_20,triple-helix,KL39:1.0 ; KL123:0.999 ; KL22:0.996 ; KL114:0....,"KL36,KL39,KL70",KL105:1.0 ; KL149:0.999 ; KL22:0.998 ; KL106:0...,"KL52,KL36,KL70",KL47:1.0 ; KL116:1.0 ; KL105:1.0 ; KL39:1.0 ; ...,"KL39,KL70",No_hits,0,No_hits,0,No_hits,0,"KL39,KL31,KL70,KL27,KL21,KL56,KL69,KL52,KL65,K..."
2,K32PH164C1,K32PH164C1__cds_20,triple-helix,KL36:1.0 ; KL39:0.998 ; KL25:0.992 ; KL116:0.9...,0,KL36:1.0 ; KL105:0.998 ; KL8:0.998 ; KL22:0.99...,0,KL36:1.0 ; KL166:1.0 ; KL46:1.0 ; KL105:1.0 ; ...,0,No_hits,0,No_hits,0,No_hits,0,"KL32,KL56"
3,K18PH07C1,K18PH07C1__cds_245,right-handed beta-helix,KL3:0.998 ; KL63:0.984 ; KL43:0.94 ; KL145:0.9...,0,KL3:0.93 ; KL63:0.872 ; KL70:0.768 ; KL54:0.64...,0,KL6:1.0 ; KL63:0.999 ; KL26:0.988 ; KL31:0.974...,0,KL63:0.594 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,0,KL6:0.523 ; KL63:0.682,0,K63,0,KL18
4,K13PH07C1L,K13PH07C1L__cds_11,right-handed beta-helix,KL23:0.992 ; KL10:0.99 ; KL30:0.975 ; KL60:0.9...,KL13,KL12:0.999 ; KL128:0.991 ; KL105:0.962 ; KL36:...,KL13,KL105:1.0 ; KL128:0.995 ; KL30:0.995 ; KL36:0....,KL13,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL13,No_associations,0,K2,0,KL13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,PFOEGONH,PFOEGONH_00078,right-handed beta-helix,KL3:1.0 ; KL81:0.985 ; KL14:0.957 ; KL52:0.857...,KL3,KL3:0.999 ; KL117:0.951 ; KL31:0.789 ; KL52:0....,KL3,KL3:1.0 ; KL67:0.99 ; KL14:0.862 ; KL52:0.794 ...,KL3,KL35:0.736 ; KL3:0.674 ; KL124:0.332 ; KL37:0....,KL3,KL35:0.71 ; KL3:0.947,KL3,K3,KL3,KL3
235,NBNDMPCG,NBNDMPCG_00163,right-handed beta-helix,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...,KL2,KL12:0.998 ; KL13:0.929 ; KL9:0.92 ; KL2:0.652...,KL2,KL12:0.999 ; KL9:0.99 ; KL13:0.981 ; KL142:0.9...,KL35,KL2:0.684 ; KL52:0.239 ; KL37:0.235 ; KL164:0....,KL2,KL2:0.829 ; KL52:0.774,KL2,K2,KL2,"KL2,KL35"
236,NJHLHPIG,NJHLHPIG_00061,right-handed beta-helix,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....,0,KL128:0.999 ; KL46:0.999 ; KL12:0.984 ; KL56:0...,0,KL46:0.999 ; KL56:0.998 ; KL128:0.984 ; KL26:0...,0,KL46:0.937 ; KL18:0.522 ; KL37:0.235 ; KL164:0...,0,KL46:0.82,0,K46,0,KL35
237,HIIECEMK,HIIECEMK_00054,right-handed beta-helix,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....,KL2,KL12:0.991 ; KL18:0.976 ; KL60:0.93 ; KL70:0.7...,0,KL8:0.999 ; KL12:0.998 ; KL18:0.997 ; KL142:0....,0,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL2,No_associations,0,K2,KL2,KL2


> Functions: 

In [266]:
# Useful functions: 

def make_labels(pred_df, predictor = "tropigat" , top_n = 40, to_ignore = []) :
    dico_pred = {"tropigat" : "TropiGAT_prediction" ,
                 "tropiseq" : "TropiSEQ_prediction", 
                 "spikehunter" : "SpikeHunter_prediction", 
                 "tropigat_new" : "TropiGAT_new_prediction", 
                 "tropilr" : "TropiLogReg_prediction", 
                 "tropisage" : "TropiSAGE_prediction"}
    col = dico_pred[predictor]
    labels_tropigat = {}
    all_kl_types = set(kltype.strip() for row in results_bench_df["Targets"] for kltype in row.split(","))
    count_kltypes = {}
    dico_prophage_count = all_kl_types
    for kltype in tqdm(dico_prophage_count) : 
        n = 0
        pred_labels , real_labels = [] , []
        for author in dico_matrices :
            if kltype in dico_matrices[author]["pool"] : 
                matrix = dico_matrices[author]["matrix"]
                for phage in matrix["Phages"].unique() :
                    #if phage not in to_ignore : 
                    if phage not in to_ignore and phage in pred_df["phage"].tolist(): 
                        top_predictions = set()
                        predictions = pred_df[pred_df["phage"] == phage][col].values
                        for calls in predictions : 
                            if predictor == "spikehunter" :
                                predicted_kltypes = [x.split(":")[0].strip().replace("K", "KL") for x in calls.split(";")]
                            else :
                                predicted_kltypes = [x.split(":")[0].strip() for x in calls.split(";")]
                            top_predictions.update(predicted_kltypes[0:top_n-1])
                        if kltype in top_predictions : 
                            pred_labels.append(1)
                        else : 
                            pred_labels.append(0)
                        if kltype in dico_hits[phage] : 
                            real_labels.append(1)
                            n += 1 
                        else :
                            real_labels.append(0)
        labels_tropigat[kltype] = {"y_pred" : pred_labels, "real_labels" : real_labels}
        sorted_dict = dict(sorted(labels_tropigat.items(), key=lambda item: int(item[0].split("KL")[1])))
        count_kltypes[kltype] = n 
    return sorted_dict , labels_tropigat, count_kltypes

def decript_dic(sorted_dict, labels_tropigat, count_kltypes, print_df=True):
    # Initialize lists to collect metrics when print_df is False
    kltypes, counts, f1_scores, accuracies, recalls, precisions, aucs, mccs = ([] for _ in range(8))
    
    if print_df:
        print("KL type", "Count", "F1", "Accuracy", "Recall", "Precision", "AUC", "MCC", sep="\t")
    
    for kltype in sorted_dict:
        if len(labels_tropigat[kltype]["real_labels"]) > 1:
            if Counter(labels_tropigat[kltype]["y_pred"])[1] == 0:
                if print_df:
                    print(kltype, count_kltypes[kltype], "error", sep="\t")
                else:
                    # Append NaN for each metric when Counter condition is met
                    kltypes.append(kltype)
                    counts.append(count_kltypes[kltype])
                    f1_scores.append(np.nan)
                    accuracies.append(np.nan)
                    recalls.append(np.nan)
                    precisions.append(np.nan)
                    aucs.append(np.nan)
                    mccs.append(np.nan)
            else:
                try:
                    # Calculate each metric
                    f1 = f1_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    precision = precision_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    recall = recall_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    mcc = matthews_corrcoef(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    accuracy = accuracy_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    auc = roc_auc_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    
                    if print_df:
                        print(kltype, count_kltypes[kltype], round(f1, 5), round(accuracy, 5), round(recall, 5), 
                              round(precision, 5), round(auc, 5), round(mcc, 5), sep="\t")
                    else:
                        # Append each metric to the corresponding list
                        kltypes.append(kltype)
                        counts.append(count_kltypes[kltype])
                        f1_scores.append(f1)
                        accuracies.append(accuracy)
                        recalls.append(recall)
                        precisions.append(precision)
                        aucs.append(auc)
                        mccs.append(mcc)                        
                except ValueError:
                    if print_df:
                        print(kltype, "error", sep="\t")
                    else:
                        # Append NaN values when a ValueError occurs
                        kltypes.append(kltype)
                        counts.append(count_kltypes[kltype])
                        f1_scores.append(np.nan)
                        accuracies.append(np.nan)
                        recalls.append(np.nan)
                        precisions.append(np.nan)
                        aucs.append(np.nan)
                        mccs.append(np.nan)

    # Return lists when print_df is False
    if not print_df:
        dataframe = pd.DataFrame({
            "KL type": kltypes,
            "Count": counts,
            "F1": f1_scores,
            "Accuracy": accuracies,
            "Recall": recalls,
            "Precision": precisions,
            "AUC": aucs,
            "MCC": mccs
        })
        dataframe = dataframe[dataframe["Count"] > 0]
        return dataframe

    
def get_calls_metrics(df , top_n = 15, to_ignore = []) :
    from collections import defaultdict
    
    n_cols_interest = 0
    # Make list of the predictions col names :
    cols_predictors = []
    for col in df.columns: 
        if col.count("good_calls") > 0 :
            cols_predictors.append(col)
    good_calls_dico = defaultdict(list)
    # iterate through the predictions : 
    for predictor in cols_predictors :
        good_calls_list = df[predictor].tolist()
        for phage, good_calls in zip(df["phage"].tolist(), good_calls_list) :
            if good_calls != "0" :
                for call in good_calls.split(",") :
                    a = (phage, call)
                    good_calls_dico[predictor].append(a)
    return good_calls_dico
                
        

In [269]:
good_calls = get_calls_metrics(results_bench_df)

for predictor in good_calls :
    print(predictor, len(good_calls[predictor]))

TropiGAT_good_calls 134
TropiGAT_new_good_calls 90
TropiSAGE_good_calls 131
TropiSEQ_good_calls 97
TropiLogReg_good_calls 21
SpikeHunter_good_calls 44


In [256]:
# Full dataset: 
sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat = make_labels(results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n = make_labels(results_bench_df, predictor = "tropigat_new", top_n = 40)
sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage = make_labels(results_bench_df, predictor = "tropisage", top_n = 40)

sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq = make_labels(results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr = make_labels(results_bench_df, predictor = "tropilr", top_n = 40)
sorted_dic_tropish, labels_tropish, count_kltypes_sh = make_labels(results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_full_df = decript_dic(sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat, print_df=False)
tropigat_n_full_df = decript_dic(sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n, print_df=False)
tropisage_full_df = decript_dic(sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage, print_df=False)

tropiseq_full_df = decript_dic(sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq, print_df=False)
tropilr_full_df = decript_dic(sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr, print_df=False)
tropish_full_df = decript_dic(sorted_dic_tropish, labels_tropish, count_kltypes_sh, print_df=False)

tropigat_full_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropigat_n_full_df.to_csv(f"{path_benchmark}/TropiGAT_new.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropisage_full_df.to_csv(f"{path_benchmark}/TropiSAGE.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

tropiseq_full_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropilr_full_df.to_csv(f"{path_benchmark}/TropiLogReg.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropish_full_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 100.81it/s]


> Comparative metrics, full: 

In [257]:
comparative_full_df = pd.DataFrame({
            "KL type": tropigat_full_df["KL type"],
            "Count": tropigat_full_df["Count"],
            # recall
            "Recall_TropiGAT": tropigat_full_df["Recall"].round(3),
            "Recall_TropiGAT_n": tropigat_n_full_df["Recall"].round(3),
            "Recall_TropiSAGE": tropisage_full_df["Recall"].round(3),
            "Recall_TropiSEQ": tropiseq_full_df["Recall"].round(3),
            "Recall_TropiLogReg": tropilr_full_df["Recall"].round(3),
            "Recall_TropiSH": tropish_full_df["Recall"].round(3),
            # precision: 
            "Precision_TropiGAT": tropigat_full_df["Precision"].round(3),
            "Precision_TropiGAT_n": tropigat_n_full_df["Precision"].round(3),
            "Precision_TropiSAGE": tropisage_full_df["Precision"].round(3),
            "Precision_TropiLogReg": tropilr_full_df["Precision"].round(3),
            "Precision_TropiSEQ": tropiseq_full_df["Precision"].round(3),
            "Precision_TropiSH": tropish_full_df["Precision"].round(3),
            # MCC: 
            "MCC_TropiGAT": tropigat_full_df["MCC"].round(3),
            "MCC_TropiGAT_n": tropigat_n_full_df["MCC"].round(3),
            "MCC_TropiSAGE": tropisage_full_df["MCC"].round(3),
            "MCC_TropiLogReg": tropilr_full_df["MCC"].round(3),
            "MCC_TropiSEQ": tropiseq_full_df["MCC"].round(3),
            "MCC_TropiSH": tropish_full_df["MCC"].round(3),
        })

comparative_full_df = comparative_full_df.fillna("-")
comparative_full_df.to_csv(f"{path_benchmark}/Comparative.Full.tsv", sep = "\t", index = False, header = True)


In [258]:
comparative_full_df

,KL type,Count,Recall_TropiGAT,Recall_TropiGAT_n,Recall_TropiSAGE,Recall_TropiSEQ,Recall_TropiLogReg,Recall_TropiSH,Precision_TropiGAT,Precision_TropiGAT_n,Precision_TropiSAGE,Precision_TropiLogReg,Precision_TropiSEQ,Precision_TropiSH,MCC_TropiGAT,MCC_TropiGAT_n,MCC_TropiSAGE,MCC_TropiLogReg,MCC_TropiSEQ,MCC_TropiSH
0,KL1,3,0.0,0.0,0.0,-,-,0.0,0.0,0.0,0.0,-,-,0.0,-0.082,-0.034,-0.017,-,-,-0.017
1,KL2,16,0.375,0.125,0.125,0.312,0.125,0.25,0.286,1.0,0.222,1.0,0.714,0.667,0.213,0.333,0.079,0.333,0.428,0.362
2,KL3,12,0.333,0.25,0.25,0.25,0.25,0.25,0.571,1.0,1.0,0.75,1.0,1.0,-0.043,0.343,0.343,0.153,0.343,0.343
3,KL4,7,-,0.286,1.0,-,-,-,-,0.095,0.226,-,-,-,-,0.038,0.402,-,-,-
4,KL5,1,0.0,-,0.0,-,-,-,0.0,-,0.0,-,-,-,-0.096,-,-0.036,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,KL144,2,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
85,KL149,2,0.0,0.0,0.0,0.0,-,-,0.0,0.0,0.0,-,0.0,-,-0.061,-0.123,-0.101,-,-0.042,-
86,KL151,1,-,-,-,1.0,-,1.0,-,-,-,-,1.0,1.0,-,-,-,-,1.0,1.0
87,KL157,1,-,1.0,1.0,1.0,1.0,1.0,-,1.0,0.083,1.0,1.0,1.0,-,1.0,0.237,1.0,1.0,1.0


> Comparative metrics, helices:

In [197]:
# Helices:  
helices_results_bench_df = results_bench_df[results_bench_df["Folds"].isin(["triple-helix","right-handed beta-helix"])]

sorted_dic_tropigat_helices, labels_tropigat_helices, count_kltypes_tropigat_helices = make_labels(helices_results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropiseq_helices, labels_tropiseq_helices, count_kltypes_tropiseq_helices = make_labels(helices_results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropish_helices, labels_tropish_helices, count_kltypes_sh_helices = make_labels(helices_results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_helices_df = decript_dic(sorted_dic_tropigat_helices, labels_tropigat_helices, count_kltypes_tropigat_helices, print_df=False)
tropiseq_helices_df = decript_dic(sorted_dic_tropiseq_helices, labels_tropiseq_helices, count_kltypes_tropiseq_helices, print_df=False)
tropish_helices_df = decript_dic(sorted_dic_tropish_helices, labels_tropish_helices, count_kltypes_sh_helices, print_df=False)

tropigat_helices_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)
tropiseq_helices_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)
tropish_helices_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)


# Full dataset: 
sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat = make_labels(results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n = make_labels(results_bench_df, predictor = "tropigat_new", top_n = 40)
sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage = make_labels(results_bench_df, predictor = "tropisage", top_n = 40)

sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq = make_labels(results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr = make_labels(results_bench_df, predictor = "tropilr", top_n = 40)
sorted_dic_tropish, labels_tropish, count_kltypes_sh = make_labels(results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_full_df = decript_dic(sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat, print_df=False)
tropigat_n_full_df = decript_dic(sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n, print_df=False)
tropisage_full_df = decript_dic(sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage, print_df=False)

tropiseq_full_df = decript_dic(sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq, print_df=False)
tropilr_full_df = decript_dic(sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr, print_df=False)
tropish_full_df = decript_dic(sorted_dic_tropish, labels_tropish, count_kltypes_sh, print_df=False)

tropigat_full_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropigat_n_full_df.to_csv(f"{path_benchmark}/TropiGAT_new.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropisage_full_df.to_csv(f"{path_benchmark}/TropiSAGE.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

tropiseq_full_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropilr_full_df.to_csv(f"{path_benchmark}/TropiLogReg.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropish_full_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 95.52it/s]
/media/concha-eloko/Linux/conda_envs/torch_geometric/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [220]:
comparative_helices_df = pd.DataFrame({
            "KL type": tropigat_helices_df["KL type"],
            "Count": tropigat_helices_df["Count"],
            # recall
            "Recall_TropiGAT": tropigat_helices_df["Recall"].round(3),
            "Recall_TropiSEQ": tropiseq_helices_df["Recall"].round(3),
            "Recall_TropiSH": tropish_helices_df["Recall"].round(3),
            # precision: 
            "Precision_TropiGAT": tropigat_helices_df["Precision"].round(3),
            "Precision_TropiSEQ": tropiseq_helices_df["Precision"].round(3),
            "Precision_TropiSH": tropish_helices_df["Precision"].round(3),
            # MCC: 
            "MCC_TropiGAT": tropigat_helices_df["MCC"].round(3),
            "MCC_TropiSEQ": tropiseq_helices_df["MCC"].round(3),
            "MCC_TropiSH": tropish_helices_df["MCC"].round(3),
        })

comparative_helices_df = comparative_helices_df.fillna("-")
comparative_helices_df.to_csv(f"{path_benchmark}/Comparative.Helices.tsv", sep = "\t", index = False, header = True)

In [221]:
comparative_helices_df

,KL type,Count,Recall_TropiGAT,Recall_TropiSEQ,Recall_TropiSH,Precision_TropiGAT,Precision_TropiSEQ,Precision_TropiSH,MCC_TropiGAT,MCC_TropiSEQ,MCC_TropiSH
0,KL1,3,0.0,-,0.0,0.0,-,0.0,-0.085,-,-0.018
1,KL2,11,0.545,0.273,0.364,0.286,1.0,0.667,0.305,0.503,0.455
2,KL3,7,0.571,0.429,0.429,0.571,1.0,1.0,0.143,0.522,0.522
3,KL4,2,-,-,-,-,-,-,-,-,-
4,KL5,1,0.0,-,-,0.0,-,-,-0.027,-,-
...,...,...,...,...,...,...,...,...,...,...,...
83,KL140,3,-,-,1.0,-,-,1.0,-,-,1.0
85,KL149,2,0.0,-,-,0.0,-,-,-0.065,-,-
86,KL151,1,-,1.0,1.0,-,1.0,1.0,-,1.0,1.0
87,KL157,1,-,-,1.0,-,-,1.0,-,-,1.0


***
### Experimentally validated depolymerase results: 

In [225]:
# regular filtration: 
tropigat_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_prediction"])
tropigat_new_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_new_prediction"])
tropigat_no_heads_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT_no_heads.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_no_heads_prediction"])


tropisage_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiSAGE.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSAGE_prediction"])

tropiseq_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiSEQ.p_05.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSEQ_prediction"])
tropilr_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiLR_prediction"])

# Ultra filtration: 
tropigat_new_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiGAT_UF.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_UF_prediction"])
tropisage_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiSAGE_UF.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSAGE_UF_prediction"])

tropiseq_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiSEQ_UF.p_05.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSEQ_UF_prediction"])
#tropilr_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "prediction"])

# SpikeHunter
SH_pred_xp_df = pd.read_csv(f"{path_benchmark}/SpikeHunter_predictions.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "SpikeHunter_prediction"])
SH_pred_xp_df["SpikeHunter_prediction"] = SH_pred_xp_df["SpikeHunter_prediction"].apply(lambda x : x.replace("K", "KL"))


> Add metadata: 

In [229]:
info_depo = pd.read_csv(f"{path_benchmark}/exp_val_depolymerase.tsv", sep = "\t", header = 0)


In [234]:
info_depo

,Phage,protein,Target
0,Klebsiella phage NTUH-K2044-K1-1,AB716666.1_prot_BAP15736.1_24,ND
1,Klebsiella phage NTUH-K2044-K1-1,AB716666.1_prot_BAP15746.1_34,K1
2,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02835.1_55,K11*
3,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02836.1_56,KN4†
4,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02837.1_57,K21*
5,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02838.1_58,KN5*
6,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02839.1_59,K25†
7,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02840.1_60,K35†
8,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02841.1_61,K1*
9,Klebsiella Phage ΦK64-1,AB897757.1_prot_BAQ02842.1_62,K64*


In [237]:
merged_df = tropigat_pred_xp_df.merge(tropigat_new_pred_xp_df, on='protein', how='inner').merge(tropisage_pred_xp_df, on='protein', how='inner').merge(tropiseq_pred_xp_df, on='protein', how='inner').merge(tropigat_new_pred_xp_df_uf, on='protein', how='inner').merge(tropisage_pred_xp_df_uf, on='protein', how='inner').merge(tropiseq_pred_xp_df_uf, on='protein', how='inner').merge(SH_pred_xp_df, on='protein', how='inner').merge(tropigat_no_heads_pred_xp_df, on='protein', how='inner')

missing_protein = merged_df[~merged_df["protein"].isin(list(info_depo["protein"]))]["protein"]
merged_df = merged_df[merged_df["protein"].isin(list(info_depo["protein"]))]
target_col = []
phage_col = []

for _, row in merged_df.iterrows() :
    try : 
        target_col.append(info_depo[info_depo["protein"] == row["protein"]]["Target"].values[0])
        phage_col.append(info_depo[info_depo["protein"] == row["protein"]]["Phage"].values[0])
    except IndexError :
        print(row["protein"])


merged_df["Target"] = target_col
merged_df["Phage"] = phage_col

'''merged_df = merged_df[['Phage','protein', 'TropiGAT_prediction', 'TropiGAT_UF_prediction_x',
       'TropiSAGE_prediction', 'TropiSEQ_UF_prediction_x',
       'TropiGAT_UF_prediction_y', 'TropiSAGE_UF_prediction',
       'TropiSEQ_UF_prediction_y', 'SpikeHunter_prediction', 'Target']]'''



"merged_df = merged_df[['Phage','protein', 'TropiGAT_prediction', 'TropiGAT_UF_prediction_x',\n       'TropiSAGE_prediction', 'TropiSEQ_UF_prediction_x',\n       'TropiGAT_UF_prediction_y', 'TropiSAGE_UF_prediction',\n       'TropiSEQ_UF_prediction_y', 'SpikeHunter_prediction', 'Target']]"

In [238]:
missing_protein

15         MT197175.1_prot_QIW86415.1_45
16         MT197175.1_prot_QIW86419.1_49
17         MT197175.1_prot_QIW86428.1_58
18         MT197176.1_prot_QJI52618.1_44
19         MT197176.1_prot_QJI52623.1_49
20         MT197176.1_prot_QJI52632.1_58
21         MT542697.1_prot_QKY78347.1_38
22         MT542697.1_prot_QKY78353.1_44
28         MZ322895.1_prot_QWY13631.1_35
29         MZ571827.1_prot_UEW68146.1_56
30         MZ571827.1_prot_UEW68150.1_60
31         MZ571831.1_prot_UEW68236.1_80
32         MZ571832.1_prot_UEP19662.1_17
33         MZ571832.1_prot_UEP19667.1_22
34          MZ571833.1_prot_UEW68243.1_1
35          MZ571834.1_prot_UEP19705.1_4
36         MZ612130.1_prot_QYC51043.1_10
37         MZ826764.1_prot_UCR74082.1_30
38         MZ826764.1_prot_UCR74083.1_31
39         MZ826764.1_prot_UCR74084.1_32
40         MZ826764.1_prot_UCR74085.1_33
41    NC_013649.2_prot_YP_003347651.1_57
47         OM032871.1_prot_UKL59188.1_41
48         ON881905.1_prot_UTN90143.1_15
Name: protein, d

In [232]:
def get_target_rank(df):
    """
    This function calculates the rank of targets in predictions for each column in the dataframe.
    Inputs:
        - df: pandas DataFrame containing prediction columns and a 'Target' column.
    Outputs:
        - Dictionary with predictors as keys and target ranks as values.
    """
    from collections import defaultdict

    final_results = {}
    weird_char = ["*", "†"]
    targets_col = df['Target'].tolist()
    assert len(targets_col) == len(df), "Targets and predictions length mismatch!"

    # Identify prediction columns
    predictors = [col for col in df.columns if "prediction" in col]

    for predictor in predictors:
        dico_ranks = defaultdict(list)
        col_pred = df[predictor].tolist()

        for pred_list, target in zip(col_pred, targets_col):
            # Clean the target
            for char in weird_char:
                target = target.replace(char, "")
            if target == "ND" or "KN" in target:
                continue

            target_list = [
                t.replace("K", "KL") if "KL" not in t else t
                for t in target.split(",")
            ]

            # Check predictions
            if pred_list == "No_hits":
                for target in target_list:
                    dico_ranks[target].append("-")
            else:
                pred_ranked = [
                    pred.split(":")[0].strip() for pred in pred_list.split(";")
                ]
                for target in target_list:
                    if target in pred_ranked:
                        rank = pred_ranked.index(target) + 1  # 1-based index
                        dico_ranks[target].append(rank)
                    else:
                        dico_ranks[target].append("-")

        # Store ranks for the current predictor
        final_results[predictor] = dict(dico_ranks)

    return final_results

In [233]:
get_target_rank(merged_df)

{'TropiGAT_prediction': {'KL1': ['-', '-', '-', '-'],
  'KL11': ['-'],
  'KL21': ['-'],
  'KL25': ['-'],
  'KL35': ['-'],
  'KL64': [4, 3],
  'KL30': [4],
  ' KL69': ['-'],
  'KL47': [1, 1],
  'KL2': ['-'],
  'KL63': ['-']},
 'TropiGAT_new_prediction': {'KL1': ['-', '-', '-', '-'],
  'KL11': ['-'],
  'KL21': ['-'],
  'KL25': ['-'],
  'KL35': ['-'],
  'KL64': [7, 1],
  'KL30': [4],
  ' KL69': ['-'],
  'KL47': [1, 1],
  'KL2': ['-'],
  'KL63': ['-']},
 'TropiSAGE_prediction': {'KL1': ['-', '-', '-', '-'],
  'KL11': ['-'],
  'KL21': ['-'],
  'KL25': ['-'],
  'KL35': [3],
  'KL64': [16, 13],
  'KL30': [9],
  ' KL69': ['-'],
  'KL47': [1, 1],
  'KL2': ['-'],
  'KL63': ['-']},
 'TropiSEQ_prediction': {'KL1': ['-', '-', '-', '-'],
  'KL11': [8],
  'KL21': ['-'],
  'KL25': ['-'],
  'KL35': [1],
  'KL64': [1, 1],
  'KL30': [1],
  ' KL69': ['-'],
  'KL47': [1, 1],
  'KL2': ['-'],
  'KL63': [1]},
 'TropiGAT_UF_prediction': {'KL1': ['-', '-', '-', '-'],
  'KL11': ['-'],
  'KL21': ['-'],
  'KL25': 

In [239]:
merged_df.to_csv(f"{path_benchmark}/Metrics.exp_val_depolymerase.raw.tsv", sep = "\t", header = True, index = False)